In [83]:
import os
import json
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import gensim

In [84]:
normalized_dir = "../polarities/normalized"

In [86]:
subreddits = list()

In [88]:
subreddit_polarities = dict()
for subreddit in os.listdir(normalized_dir):
    if subreddit.split('.')[-1] == 'json':
        if subreddit.split('_standardized')[0] != 'BingeEatingRecovery':
            with open(os.path.join(normalized_dir,subreddit)) as f:
                subreddit_polarities[subreddit.split('_standardized')[0]] = json.load(f)
                subreddits.append(subreddit.split('_standardized')[0])

In [89]:
subreddits

['AnorexiaRecovery',
 'bingeeating',
 'eating_disorders',
 'EdAnonymousAdults',
 'EatingDisorderHope',
 'selftext',
 'bulimia',
 'BulimiaRecovery',
 'EDRecovery_public',
 'BingeEatingDisorder',
 'EatingDisorders',
 'anorexiaflareuphelp',
 'edsupport',
 'PurgingDisorder',
 'AnorexiaNervosa',
 'BulimiaAndAnaSupport',
 'EDAnonymous',
 'NotOtherwiseSpecified']

In [90]:
common = list(subreddit_polarities[subreddits[0]].keys())
for subreddit in subreddits:
#     print(subreddit)
    common = [value for value in common if value in subreddit_polarities[subreddit].keys()]

In [92]:
df_list = list()
for word in common:
    for subreddit in subreddits:
        df_list.append({'subreddit': subreddit, 'word': word, 'polarity': subreddit_polarities[subreddit][word]})

In [93]:
shared_words = pd.DataFrame.from_dict(df_list)

In [94]:
shared_words

,subreddit,word,polarity
0,AnorexiaRecovery,want,-0.319114
1,bingeeating,want,0.147042
2,eating_disorders,want,-2.114622
3,EdAnonymousAdults,want,-7.227650
4,EatingDisorderHope,want,-1.934358
...,...,...,...
769,PurgingDisorder,know,-0.042790
770,AnorexiaNervosa,know,-1.111143
771,BulimiaAndAnaSupport,know,4.377576
772,EDAnonymous,know,-4.385993


In [95]:
sqlite_path = "../reddit_ed_0.6.0.sqlite3"
engine = create_engine(
    f"sqlite:///{sqlite_path}",
    connect_args={"check_same_thread":False},
)

In [96]:
query = f"select subreddit, count(*) as count from subreddit_submission_metadata group by subreddit;"
subreddits = pd.read_sql_query(text(query),con=engine.connect())

In [97]:
merged_df = shared_words.merge(subreddits,how='inner',on='subreddit')

In [99]:
merged_df[merged_df['word']=="food"].sort_values(by='polarity')

,subreddit,word,polarity,count
165,EdAnonymousAdults,food,-5.284351,4954
681,EDAnonymous,food,-3.306161,3910
251,bulimia,food,-1.825423,2326
122,eating_disorders,food,-0.839858,3663
208,EatingDisorderHope,food,-0.815537,99
638,BulimiaAndAnaSupport,food,-0.680637,117
380,BingeEatingDisorder,food,-0.259065,393
423,EatingDisorders,food,-0.226930,904
509,edsupport,food,-0.205861,83
79,bingeeating,food,-0.181133,421
